## Simulate Sweep AI Using EvaDB

### Introduction
Sweep is an AI tool that helps transform feature requests into actual code changes. In this notebook, we will demonstrate how to use the ChatGPT calls from EvaDB to implement one of the basic functions of Sweep AI. We will modify a Github repository remotely using EvaDB.

### Setup

In [1]:
!apt install postgresql
!service postgresql start
!sudo -u postgres psql -c "CREATE USER yiding WITH SUPERUSER PASSWORD '6422db'"
!sudo -u postgres psql -c "CREATE DATABASE evadb2"
%pip install --quiet "evadb[document]"
%pip install psycopg2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 9 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64

### Connect EvaDB to Database

In [2]:
import evadb
cursor=evadb.connect().cursor()
params = {
    "user": "yiding",
    "password": "6422db",
    "host": "localhost",
    "port": "5432",
    "database": "evadb2",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 754kB/s] 
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


,0
0,The database postgres_data has been successful...


### Create Database Table

In [3]:
cursor.query("""USE postgres_data{
  DROP TABLE IF EXISTS github
}""").df()

,status
0,success


In [4]:
cursor.query("""USE postgres_data{
  CREATE TABLE github(
    name VARCHAR(10),
    request VARCHAR(200)
  )
}""").df()

,status
0,success


### Create Requests

We now create several example requests and import them into the database for future use.

In [5]:
query = """
USE postgres_data {
  INSERT INTO github (name, request) VALUES ('request-1',
  'Write a python function that checks if the given string is a palindrome. The function
  takes in one parameter, which is the input word.')
}
"""
cursor.query(query).df()

,status
0,success


In [6]:
query = """
USE postgres_data {
  INSERT INTO github (name, request) VALUES ('request-2',
  'Write a python function that checks if a given list of integers is strictly decreasing.
  The function takes in one parameter, which is the input list.')
}
"""
cursor.query(query).df()

,status
0,success


In [7]:
requests=cursor.query("SELECT * FROM postgres_data.github;").df()

/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

In [8]:
file_names=requests["name"]
prompts=requests["request"]

### Set up OpenAI

We now use the unique API key to connect to the GPT service from OpenAI.

In [9]:
! pip install --upgrade openai
import re
import os
from openai import AsyncOpenAI

In [10]:
client = AsyncOpenAI(api_key="YOUR OPENAI API KEY HERE")
code=[]
for i in range(0,len(prompts)):
  prompt="Only reply the Python code. "+prompts[i]
  completion = await client.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}])
  code.append(completion.choices[0].message.content)

In [11]:
for i in range(0,len(prompts)):
  fname=file_names[i]+".py"
  content=code[i]
  with open(fname,"w") as file:
    file.write(content)

### Connect to GitHub

Now we connect to the GitHub repository that we want to put our files on.

In [12]:
!pip install PyGithub
from github import Github
from github import InputGitTreeElement

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 31.8 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


In [13]:
github_token = 'YOUR GITHUB TOKEN HERE'
repo_owner = 'YOUR GITHUB USERNAME HERE'
repo_name = 'evadb2'
branch_name = 'BRANCH NAME HERE'

In [14]:
g=Github(github_token)
for i in range(0,len(prompts)):
  file_path=file_names[i]+".py"
  new_content=code[i]
  commit_message="Finish "+file_names[i]
  repo=g.get_repo(f"{repo_owner}/{repo_name}")
  repo_branch=repo.get_branch(branch_name)
  commit=repo_branch.commit
  new_blob=repo.create_git_blob(new_content,"utf-8")
  tree=repo.get_git_tree(commit.sha)
  new_tree = repo.create_git_tree(
      [
          InputGitTreeElement(
              path=file_path,
              mode='100644',
              type='blob',
              sha=new_blob.sha
          )
      ],
      base_tree=tree
  )
  new_commit = repo.create_git_commit(
      message=commit_message,
      tree=repo.get_git_tree(sha=new_tree.sha),
      parents=[repo.get_git_commit(repo.get_branch("main").commit.sha)]
  )
  ref=repo.get_git_ref(ref="heads/main")
  ref.edit(sha=new_commit.sha)